In [152]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [153]:
url = "https://finance.naver.com/sise/"

page = urlopen(url)
soup = BeautifulSoup(page, 'html.parser')

In [154]:
soup.title

<title>국내증시 : 네이버 금융</title>

In [155]:
soup.find('ul',id='popularItemList').find_all('li')[0]

<li><em>1.</em><a href="/item/main.nhn?code=019170" onclick="clickcr(this,'boa.list','019170','1',event)">신풍제약</a><span class="up">95,600</span><em class="bu_p bu_pup"><span class="blind">상승</span></em></li>

In [156]:
### 인기 검색 종목 부분

pop = soup.find('ul',id='popularItemList')
pop_list = pop.find_all('li')

stock_names = []
stock_current_prices = []
up_downs = []

for one in pop_list:
    stock_name = one.find('a').text
    stock_names.append(stock_name) # 종목명
    
    stock_price = one.find('span').text
    stock_current_prices.append(stock_price) # 종목가격
    
    up_down = one.find('span', class_='blind').text
    up_downs.append(up_down) # 상승/하락
    
#     code = one.find('a')['href'][-6:]
#     base_url='https://finance.naver.com/item/main.nhn?code='+code
    
#     page2 = urlopen(base_url)
#     soup2 = BeautifulSoup(page2, 'html.parser')
#     print(soup2.title)
    
#     soup2.


In [157]:
import pandas as pd

In [158]:
dat = {'종목명':stock_names,'현재가':stock_current_prices, '상하여력':up_downs}
dat = pd.DataFrame(dat)
dat

,종목명,현재가,상하여력
0,신풍제약,"95,600",상승
1,삼성전자,"80,400",상승
2,HMM,"48,100",상승
3,카카오,"157,000",하락
4,삼성전기,"188,000",상승
5,카카오게임즈,"76,000",상승
6,진원생명과학,"50,800",상승
7,LG전자,"167,000",상승
8,현대차,"236,000",하락
9,NAVER,"410,000",하락


### Top종목

In [159]:
# top_list = soup.select('#siselist_tab_0')
# print(top_list)

In [160]:
idxs = [] # 순위
continuities = [] # 연속
accumulates = [] # 누적
top_stock_names = [] # 종목명
top_stock_name_prices = [] # 현재가
com_with_yest = [] # 전일비
fluc_rates = [] # 등락률
trad_volumes = [] # 거래량
starts = [] # 시가
highs = [] # 고가
lows = [] # 저가
 

In [161]:
top_list = soup.find('table',id='siselist_tab_0').find_all('tr')
# print(len(top_list))

for one in range(2,(len(top_list)-5)): # None 값 제거를 위해 in range를 사용하였고, 크롤링 구역을 잘 뜯어보자! 
#     num = one.find('img')               그리고 int타입에 대해 생각해보기!!!
#     print(top_list[one])
    num = top_list[one].find('img')['src'][-5:-4] 
    idxs.append(num) # 순위
#     print(idxs)  
    
    con = top_list[one].find_all('td', class_='number')[0].text
    continuities.append(con) # 연속
    
    acc = top_list[one].find_all('td', class_='number')[1].text
    accumulates.append(acc) # 누적
    
    names = top_list[one].find('a',class_='tltle').text
    top_stock_names.append(names) # 종목명
    
    price = top_list[one].find_all('td',class_='number')[2].text
    top_stock_name_prices.append(price) # 현재가
    
    com = top_list[one].find_all('td',class_='number')[3].text.split()
    com_with_yest.append(com) # 전일비
    
    flu = top_list[one].find_all('td',class_='number')[4].text.split()
    fluc_rates.append(flu) # 등락률
    
    trad = top_list[one].find_all('td',class_='number')[5].text
    trad_volumes.append(trad) # 거래량
    
    start = top_list[one].find_all('td',class_='number')[6].text
    starts.append(start) # 시가
    
    high = top_list[one].find_all('td',class_='number')[7].text
    highs.append(high) # 고가
    
    low = top_list[one].find_all('td', class_='number')[8].text
    lows.append(low) #저가


print(len(idxs))

5


In [162]:
import pandas as pd

In [163]:
dat = {'순위': idxs, '연속': continuities, '누적':accumulates, 
       '종목명':top_stock_names , '현재가':top_stock_name_prices , 
       '전일비':com_with_yest,'등락률' :fluc_rates , '거래량':trad_volumes, 
       '시가':starts ,'고가':highs, '저가':lows}
dat = pd.DataFrame(dat)
dat

,순위,연속,누적,종목명,현재가,전일비,등락률,거래량,시가,고가,저가
0,1,1,2,코리아에셋투자증권,"16,350","[상한, 3,750]",[+29.76%],"2,831,589","12,550","16,350","12,350"
1,2,1,4,한국비엔씨,"11,150","[상한, 2,570]",[+29.95%],"26,790,046","8,610","11,150","8,510"
2,3,2,2,유진스팩5호,"6,370","[상한, 1,470]",[+30.00%],"2,191,485","5,890","6,370","5,650"
3,4,1,4,하나머스트7호스팩,"4,130","[상한, 950]",[+29.87%],"12,583,776","3,315","4,130","3,265"
4,5,1,5,유진스팩6호,"4,815","[상한, 1,110]",[+29.96%],"6,173,742","3,900","4,815","3,885"


In [164]:
stock = dat.to_csv('0705_top_stock.csv',index=False)
stock = dat.to_excel('0705_top_stock.xlsx',index=False)

In [165]:
import os

In [166]:
path = os.getcwd()
print(os.chdir(path))

None
